In [1]:
%%capture
from pathlib import Path

if Path.cwd().stem == "features":
    %cd ../..
    %load_ext autoreload
    %autoreload 2

In [2]:
import logging
from pathlib import Path

import polars as pl
from polars import col

from src.data.database_manager import DatabaseManager
from src.features.scaling import scale_min_max
from src.features.transformations import (
    add_time_column,
)
from src.helpers import to_describe
from src.log_config import configure_logging

configure_logging(
    stream_level=logging.DEBUG,
    ignore_libs=["matplotlib", "Comm", "bokeh", "tornado"],
)

pl.Config.set_tbl_rows(12)  # for the 12 trials
# plt.rcParams["figure.figsize"] = [15, 5]  # default is [6, 4]

polars.config.Config

In [3]:
db = DatabaseManager()

In [4]:
with db:
    df = db.read_table("Raw_EDA")
df

trial_id,trial_number,participant_id,rownumber,timestamp,samplenumber,eda_d_battery,eda_raw,eda_d_packetreceptionrate
u16,u8,u8,u32,f64,i64,f64,f64,i64
1,1,1,37660,294210.3603,57892,3686.174359,0.752359,100
1,1,1,37661,294211.3575,57893,3693.45641,0.754579,100
1,1,1,37662,294211.3575,57894,3687.630769,0.753247,100
1,1,1,37663,294224.331,57895,3677.435897,0.753247,100
1,1,1,37664,294242.275,57896,3687.630769,0.754135,100
1,1,1,37665,294242.275,57897,3694.912821,0.752359,100
…,…,…,…,…,…,…,…,…
332,12,28,355477,2.7771e6,467074,3617.723077,13.679468,99
332,12,28,355478,2.7771e6,467075,3582.769231,13.674363,99


In [7]:
df = df.unique("timestamp").sort(["trial_id", "timestamp"])
df = add_time_column(df)

In [51]:
df = df.with_columns(
    (col("timestamp") * 1000).cast(pl.Int64).alias("timestamp_us"),
)